Import packages for entire notebook

In [ ]:
import sqlite3, json, re, datetime, math, emoji, matplotlib, calmap
print(matplotlib.get_backend())
import mplcairo
matplotlib.use("module://mplcairo.macosx")
print(matplotlib.get_backend())
print(mplcairo._mplcairo.__cairo_version__)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter, OrderedDict

Import stopwords

In [ ]:
stopwords = json.load(open('stopwords.json', 'r'))

Connect to Messages database using Sqlite3

In [ ]:
conn = sqlite3.connect('/Users/Brienna/Library/Messages/chat.db')
c = conn.cursor()

In [ ]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

In [ ]:
print(c.execute('select * from message_attachment_join').description)
cmd1 = 'SELECT * FROM chat'
c.execute(cmd1)
deleted_msg = pd.DataFrame(c.fetchall(), columns=['ROWID', 'guid', 'style', 'state', 'account_id', 'properties','chat_identifier','service_name','room_name','account_login','is_archived','last_addressed_handle','display_name','group_id','is_filtered','successful_query'])
deleted_msg

**Let user identify which handle_id to analyze**

View headings in the Messages data

In [ ]:
c.execute('select * from message')
headings = [description[0] for description in c.description]
headings

Extract the rowid, text, is_from_me, and datetime columns, placing into dataframe

In [ ]:
cmd1 = 'SELECT ROWID, text, is_from_me, \
        datetime(date + strftime(\'%s\',\'2001-01-01\'), \'unixepoch\') as date_utc \
        FROM message WHERE handle_id=47 OR handle_id=56 OR handle_id=90'
c.execute(cmd1)
df_msg = pd.DataFrame(c.fetchall(), columns=['id', 'text', 'is_from_me', 'time']).sort_values(by='time')
df_msg

Convert datatime to something useable.

In [ ]:
df_msg['time'] = [datetime.datetime.strptime(str(t), '%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours=-4) for t in df_msg['time']]
df_msg['new_date'] = [d.date() for d in df_msg['time']]
df_msg['new_time'] = [d.time() for d in df_msg['time']]
df_msg['new_hours'] = [d.hour for d in df_msg['time']]
df_msg

How long the conversation has been going

In [ ]:
start = df_msg['new_date'].iloc[0]
end = df_msg['new_date'].iloc[-1]
print('from ' + str(start) + ' until ' + str(end))

Total messages sent

In [ ]:
total = len(df_msg)
by_me = len(df_msg[df_msg['is_from_me'] == 1])
by_himher = total - by_me
print('Total: ' + str(total))
print('From me: ' + str(by_me))
print('From him/her: ' + str(by_himher))

Word frequencies

In [ ]:
frequencies = {}
for message in df_msg['text']:
    if message != None:
        words = message.split(" ")
        for word in words:
            word = re.sub(r'[^\w\s]','', word).lower().strip()
            if word not in stopwords and word != '':
                if word in frequencies:
                    frequencies[word] += 1
                else:
                    frequencies[word] = 1

frequencies_sorted = sorted(frequencies.items(), key=lambda kv: kv[1])
print(frequencies_sorted)

Most active day:

In [ ]:
most_active_day = df_msg['new_date'].mode()[0]
most_active_day.strftime('%m.%d.%Y')

Number of texts on that day:

In [ ]:
df_on_the_most_active_day = df_msg[df_msg['new_date'] == most_active_day]
num_of_texts = len(df_on_the_most_active_day)
num_of_texts_from_me = len(df_on_the_most_active_day[df_on_the_most_active_day['is_from_me'] == 1])
num_of_texts_from_them = num_of_texts - num_of_texts_from_me

print('Total texts sent on ' + str(most_common_day) + ' was ' + str(num_of_texts))
print('From me: ' + str(num_of_texts_from_me))
print('From him/her: ' + str(num_of_texts_from_them))

Average messages per day that we texted

In [ ]:
messages_total = 0;
distinct_days = 0;
last_day_tracked = None;

for index, row in df_msg.iterrows():
    message = row['text']
    if message != None:
        messages_total += 1
        current_day = row['new_date']
        if last_day_tracked != current_day: 
            distinct_days += 1
        last_day_tracked = current_day
            
print('Sent ' + str(math.floor(messages_total / distinct_days)) + ' messages on average each day.')

Create radar/spider plot showing average daily activity

In [ ]:
# Format data frames

df_24hrs_me = df_msg[df_msg['is_from_me'] == 1]['new_hours']
df_24hrs_himher = df_msg[df_msg['is_from_me'] == 0]['new_hours']
values_me = df_24hrs_me.value_counts().sort_index().values.flatten().tolist() # IMPORTANT TO SORT HOURS
values_himher = df_24hrs_himher.value_counts().sort_index().values.flatten().tolist() # IMPORTANT TO SORT HOURS

# We need to repeat the first value to close the circular graph:
values_me += values_me[:1]
values_himher += values_himher[:1]

# Get number of variables
categories = set(list(df_msg['new_hours'])[1:]) # set() reduces to distinct values
N = len(categories)

In [ ]:
# Set angle of each axis in the plot (again repeating first value to close the circular graph)
angles = [n / float(N) * 2 * math.pi for n in range(N)]
angles += angles[:1]

# Initialize spider plot
ax = plt.subplot(111, polar=True)

## If you want the first axis to be on top
ax.set_theta_offset(math.pi/2)
ax.set_theta_direction(-1)

# Draw one axe per variable + add labels 
plt.xticks(angles[:-1], categories, color='grey', size=8);

# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([1000,2000,3000,4000,5000,6000], ["1k", "2k", "3k","4k","5k","6k"], color='grey', size=8)
plt.ylim(0,max(values_me))

## ----------- Plot Individual 1 :: me
ax.plot(angles, values_me, linewidth=1, linestyle='solid')
ax.fill(angles, values_me, 'b', alpha=0.1);
 
## ----------- Plot Individual 2 :: himher
ax.plot(angles, values_himher, linewidth=1, linestyle='solid')
ax.fill(angles, values_himher, 'r', alpha=0.1)

red_patch = mpatches.Patch(color='r', label='Him',alpha=0.1)
blue_patch = mpatches.Patch(color='b', label='Me',alpha=0.1)
plt.legend(handles=[red_patch, blue_patch],loc='upper right', bbox_to_anchor=(0.1,0.1));

First occurrence of "I love you"

In [ ]:
df_msg[df_msg['text'].str.contains('i love you', case=False) == True].sort_values(by='time').head(10)


# idxmax shows first index value by condition, only necessitates that index is unique

Alternative query to avoid cases like "I love your wordplay"

In [ ]:
love_tests = pd.Series(['I love you', 'I love your wordplay']) # I don't really have other instances rn
love_tests.str.contains(r'i love you\b.*', case=False)
ilys = df_msg[df_msg['text'].str.contains(r'i love you\b.*', case=False) == True].sort_values(by='time')
print('Said "I love you" ' + str(len(ilys)) + ' times')
ilys.head(10)

In [ ]:
# The actual texts:
print(df_msg.iloc[54171].text)
print(df_msg.iloc[54172].text)

In [ ]:
messages_week = df_msg.set_index('time').resample('W-MON')['text'].count()
print(messages_week[50:100])
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
plt.plot(messages_week, label='messages', color='pink') # the 200 is where on y axis the arrow points to

fig.suptitle('Weekly message overview', fontsize=20)
plt.xlabel('Weeks', fontsize=18)
plt.ylabel('Messages', fontsize=18)
plt.annotate('Ireland', (mdates.date2num(datetime.datetime(2018, 3, 15)), 200), xytext=(-100,0), 
            textcoords='offset points', size=20,
            va='center', ha='center',
            arrowprops=dict(arrowstyle="->",
                           connectionstyle='arc3, rad=-0.2',
                           lw=2),
            )
plt.annotate('Bri in DC', (mdates.date2num(datetime.datetime(2018, 8, 28)), 205), xytext=(50, -50),
            textcoords='offset points', size=20,
            va='center', ha='center',
            arrowprops=dict(arrowstyle="->", lw=2))

Same analysis as above but with heat map (BROKEN RN) 
http://nbviewer.jupyter.org/github/home-assistant/home-assistant-notebooks/blob/master/DataExploration-2/DataExploration-2.ipynb

Build data frame for heatmap. This data frame contains dates, days of the week, and frequency of texts on that day. 

# Need to populate missing dates with zeroes

If we weren't using calmap, a way would be to use seaborn to make a calendar heatmap table. The dataframe needs to have rows that represent that week in the year (from 1 - 52), and columns that represent each day of the week. The values are how many texts were sent that day.

Can hover over squares to find date?

Calmap does this for us.
https://pythonhosted.org/calmap/

https://pythonhosted.org/calmap/_modules/calmap.html

In [ ]:
# Get message counts for each day 
msg_counts = df_msg.set_index('time').resample('D')['text'].count()
heatmap_df = pd.DataFrame({'date': msg_counts.index, 'count': msg_counts.values})

# Assign each day with Monday, Tuesday, Wednesday, etc
def label_day(date):
    day_of_week = date.weekday()
    switcher = {
        0: "Monday",
        1: "Tuesday",
        2: "Wednesday",
        3: "Thursday",
        4: "Friday",
        5: "Saturday",
        6: "Sunday"
    }
    return switcher.get(day_of_week)

heatmap_df['day'] = messages_day.index.map(label_day)
# Remove dates with 0 activity (we don't want them in the activity comparison)
heatmap_df = heatmap_df[heatmap_df['count'] != 0]

fig, axes = calmap.calendarplot(
    heatmap_df.set_index('date')['count'], 
    dayticks=[0,2,4,6],
    yearlabels = True,
    yearascending = False,
    how=None
)

plt.savefig('heatmap.png')

Shows who was more predominant in texting that day, not how many text messages were sent that day. The more toward 1, the more it was me. The more toward 0, the more it was him.

After subtracting 0.5 (the mean), the positive numbers are me. The negative numbers are him.

Note: Stopped working on this. Heatmap is probably not the best way to show this data.

In [ ]:
# Shows frequency of messages that are from me or him -- is_from_me compares who spoke the most that day
df_activity = df_msg.set_index('time').resample('D')['is_from_me'].mean()
df_activity = df_activity[df_activity != 0]
df_activity = df_activity.subtract(0.5)
print(df_activity)

df_activity = df_activity[df_activity != -0.5]

**NATURAL LANGUAGE PROCESSING**

https://github.com/guiem/my_notebooks/blob/master/anniversary/anniversary.ipynb

Generate word cloud (needs stopwords and fixing).

In [ ]:
fullTexts = ""
for message in df_msg['text']:
    if message != None:
        fullTexts += message.lower()

def generate_wordcloud(text):
    wordcloud = WordCloud(font_path = '/Library/Fonts/Verdana.ttf',
                         relative_scaling = 1.0).generate(text)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

generate_wordcloud(fullTexts)

df_with_text = df_msg.text_normalized.dropna()
top_1000 = pd.Series(' '.join(df_with_text).split()).value_counts()[:1000]
wc = WordCloud(background_color='white')
wc.generate_from_frequencies(list(top_1000).to_dict().items())
plt.figure(figsize=(20,10))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

Time to respond to a previous message

## **Emojis**

This code captures emojis that are followed by a space. Otherwise I'll need to substitute regexs for every emoji.

To get newer emojis if I use OS X Sierra: https://stephenradford.me/install-high-sierra-emoji-on-older-versions/

Bugs:
- Some emojis have two code points \u\u, and those are not recognized...? Such as the red heart.

In [ ]:
df_emojis = pd.DataFrame(columns=['emoji', 'is_from_me'])
for index, row in df_msg.iterrows():
    message = row['text']
    if message: # some messages are None?
        for word in message.split(' '):
            for char in word:
                if char in emoji.UNICODE_EMOJI:
                    df_emojis = df_emojis.append({'emoji': char, 'is_from_me': row['is_from_me']}, ignore_index=True)

In [ ]:
df_emojis

In [ ]:
frequent_emojis = Counter(df_emojis['emoji']).most_common(25)
frequent_emojis

In [ ]:
data = {}

# reiterate over df_emojis using each frequent emoji to find numbers of times we each sent it
for emoji in frequent_emojis:
    # Find all instances of the emoji
    instances = df_emojis[df_emojis['emoji'] == emoji[0]]
    # Loop over each instance to see who sent it
    for index, row in instances.iterrows():
        # Get who sent it
        who_sent_it = row['is_from_me']
        # Update frequencies
        if emoji in data:
            data[emoji][who_sent_it] += 1
        else:
            if who_sent_it == 1:
                data[emoji] = {0: 0, 1: 1}
            else:
                data[emoji] = {0: 1, 1: 0}

print(data)
print()

labels = list(zip(*data.keys()))[0]
print(labels)
print()

his_freqs = [item[0] for key, item in data.items()]
her_freqs = [item[1] for key, item in data.items()]
print(his_freqs)
print(her_freqs)

In [ ]:
import matplotlib.font_manager as mfm
emoji_font = mfm.FontProperties(fname="/System/Library/Fonts/Apple Color Emoji.ttc")

N = len(labels)
ind = np.arange(N) # x locations for the groups
width = 0.8

plt.figure(figsize=(12,8))
ax = plt.gca()
plt.title('Most Frequently Used Emojis')
plt.ylabel('Frequency', fontsize=15)
plt.xlabel('Emojis', fontsize=15)

p1 = plt.bar(ind, his_freqs, width, color="lightblue")
p2 = plt.bar(ind, her_freqs, width, bottom=his_freqs, color="pink")

plt.tick_params(
    axis='x',
    which='both',
    bottom=False,
    top=False,
    labelbottom=False
)

new_ylim = plt.ylim()[1]+30
plt.ylim(0, new_ylim)

# Make labels
for rect1, rect2, label in zip(p1, p2, labels):
    height = rect1.get_height() + rect2.get_height()
    plt.annotate(
        label,
        (rect1.get_x() + rect1.get_width()/2, height+5),
        ha="center",
        va="bottom",
        fontsize=30,
        fontproperties=emoji_font
    )

plt.legend((p1[0], p2[0]), ("Him", "Me"), fontsize=12, ncol=4, framealpha=0, fancybox=True)
    
plt.show()

Note: To Toggle Scrolling go Cell > Current Outputs > Toggle Scrolling. Or click on the cell.

Make stacked chart with my usages and his usages stacked.

And do this:
https://medium.freecodecamp.org/and-the-most-popular-developer-emoji-is-d660a9687be7

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import numpy as np

prop = FontProperties(fname='/System/Library/Fonts/Apple Color Emoji.ttc')

freqs = [301, 96, 53, 81, 42]
labels = ['😊', '😱', '😂', '😄', '😛']
plt.figure(figsize=(12,8))
p1 = plt.bar(np.arange(len(labels)), freqs, 0.8)
new_ylim = plt.ylim()[1]+30
plt.ylim(0, new_ylim)

# Make labels
for rect1, label in zip(p1, labels):
    height = rect1.get_height()
    plt.annotate(
        label,
        (rect1.get_x() + rect1.get_width()/2, height+5),
        ha="center",
        va="bottom",
        fontsize='30',
        fontproperties = prop
    )
    
plt.show()

**Emoji frequency word cloud**

In [ ]:
emoji_str = df_emojis['emoji'].str.cat(sep=' ')
try:
    emoji_str.decode('utf-8')
    print('is utf8')
except UnicodeError:
    print('not utf8')
wordcloud=WordCloud(font_path="/System/Library/Fonts/Apple Color Emoji.ttc", regexp=r"(?:[^\s])", width=500,height=500,margin=1).generate(emoji_str)
plt.imshow(wordcloud,interpolation='bilinear')

find most common abbreviation 'idk' 'smh' 'wtf' 'brb


